In [ ]:
!git clone https://github.com/Yaswanth-B/AccessibleLLM.git

In [ ]:
!pip install bert-score torch transformers

In [ ]:
!pip install rouge

In [ ]:
pip install git+https://github.com/openai/CLIP.git

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')

In [45]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import bert_score

with open("/content/AccessibleLLM/object detection/mode1d3.txt", "r") as file:
    model1_text = file.read()

blip2_text = model1_text.strip().split('\n')

with open("/content/AccessibleLLM/object detection/model2d3.txt", "r") as file:
    model2_text = file.read()

VitGpt_text = model2_text.strip().split('\n')

with open("/content/ds3text.txt", "r") as file:
    references_text = file.read()

true_text = references_text.strip().split('\n')

# Compute BERTScore for BLIP-2 Model
P_A, R_A, F1_A = bert_score.score(
    blip2_text,
    true_text,
    lang='en',  # Language of the captions (e.g., 'en' for English)
    model_type='bert-base-uncased',  # BERT model type
)

# Compute BERTScore for VitGpt Model
P_B, R_B, F1_B = bert_score.score(
    VitGpt_text,
    true_text,
    lang='en',  # Language of the captions (e.g., 'en' for English)
    model_type='bert-base-uncased',  # BERT model type
)

print("BERTScore for BLIP-2 Model:")
print(f"Precision: {P_A.mean():.4f}")
print(f"Recall: {R_A.mean():.4f}")
print(f"F1-score: {F1_A.mean():.4f}")

print("\nBERTScore for VitGpt Model:")
print(f"Precision: {P_B.mean():.4f}")
print(f"Recall: {R_B.mean():.4f}")
print(f"F1-score: {F1_B.mean():.4f}")


BERTScore for BLIP-2 Model:
Precision: 0.7260
Recall: 0.7872
F1-score: 0.7541

BERTScore for VitGpt Model:
Precision: 0.6246
Recall: 0.6585
F1-score: 0.6362


In [14]:
import nltk

def compute_gleu_score(generated_caption, reference_caption):
    gen_tokens = nltk.word_tokenize(generated_caption.lower())
    ref_tokens = nltk.word_tokenize(reference_caption.lower())
    gleu_score = nltk.translate.gleu_score.sentence_gleu([ref_tokens], gen_tokens)
    return gleu_score

def compute_average_gleu_scores(model_captions, reference_captions):
    total_gleu_score = 0.0
    num_captions = len(model_captions)

    for i in range(num_captions):
        model_caption = model_captions[i]
        ref_caption = reference_captions[i]
        gleu_score = compute_gleu_score(model_caption, ref_caption)
        total_gleu_score += gleu_score

    average_gleu_score = total_gleu_score / num_captions
    return average_gleu_score

def read_captions_from_file(file_name):
    with open(file_name, 'r') as file:
        captions = file.read().split('\n')
    return [caption.strip() for caption in captions if caption.strip()]

# Read reference captions from "truetext.txt"
reference_captions = read_captions_from_file("/content/ds3text.txt")

# Read model captions from "model1.txt" (for Model A) and "model2.txt" (for Model B)
captions_model_a = read_captions_from_file("/content/AccessibleLLM/object detection/mode1d3.txt")
captions_model_b = read_captions_from_file("/content/AccessibleLLM/object detection/model2d3.txt")

# Compute average GLEU scores for Model A and Model B
avg_gleu_model_a = compute_average_gleu_scores(captions_model_a, reference_captions)
avg_gleu_model_b = compute_average_gleu_scores(captions_model_b, reference_captions)

# Print the results
print("Average GLEU Score for BLIP-2:", avg_gleu_model_a)
print("Average GLEU Score for VItGpt:", avg_gleu_model_b)

# Determine which model performs better
if avg_gleu_model_a > avg_gleu_model_b:
    print("BLIP-2 performs better.")
elif avg_gleu_model_b > avg_gleu_model_a:
    print("VitGpt performs better.")
else:
    print("Both models perform equally well.")


Average GLEU Score for BLIP-2: 0.3281658319708012
Average GLEU Score for VItGpt: 0.15503852724900705
BLIP-2 performs better.


In [15]:
import nltk
from nltk.translate.bleu_score import corpus_bleu

def read_captions_from_file(file_name):
    with open(file_name, 'r') as file:
        captions = file.read().split('\n')
    return [caption.strip() for caption in captions if caption.strip()]

# Read reference captions from "truetext.txt"
reference_captions = read_captions_from_file("/content/ds3text.txt")

# Read model captions from "model1.txt" (for Model A) and "model2.txt" (for Model B)
captions_model_a = read_captions_from_file("/content/AccessibleLLM/object detection/mode1d3.txt")
captions_model_b = read_captions_from_file("/content/AccessibleLLM/object detection/model2d3.txt")

# Tokenize captions
model1_captions = [nltk.word_tokenize(caption.lower()) for caption in captions_model_a]
model2_captions = [nltk.word_tokenize(caption.lower()) for caption in captions_model_b]
reference_captions = [[nltk.word_tokenize(caption.lower())] for caption in reference_captions]

# Calculate BLEU score for each model
bleu_score_model1 = corpus_bleu(reference_captions, model1_captions)
bleu_score_model2 = corpus_bleu(reference_captions, model2_captions)

print("BLEU Score for BLIP-2:", bleu_score_model1)
print("BLEU Score for VitGpt:", bleu_score_model2)


BLEU Score for BLIP-2: 0.2853194240578867
BLEU Score for VitGpt: 0.09539884316244567


In [36]:
from rouge import Rouge

# Load reference captions and generated captions
with open("/content/ds3text.txt", "r") as ref_file:
    reference_captions = ref_file.readlines()

with open("/content/AccessibleLLM/object detection/mode1d3.txt", "r") as model1_file:
    model1_captions = model1_file.readlines()

with open("/content/AccessibleLLM/object detection/model2d3.txt", "r") as model2_file:
    model2_captions = model2_file.readlines()

# Initialize ROUGE
rouge = Rouge()

# Compute ROUGE scores for model 1
scores_model1 = rouge.get_scores(model1_captions, reference_captions, avg=True)
print("ROUGE Scores for BLIP-2:")
print(scores_model1)

# Compute ROUGE scores for model 2
scores_model2 = rouge.get_scores(model2_captions, reference_captions, avg=True)
print("ROUGE Scores for VitGpt:")
print(scores_model2)

# Calculate precision, recall, and F1-score based on ROUGE scores
precision_model1 = scores_model1["rouge-l"]["p"]
recall_model1 = scores_model1["rouge-l"]["r"]
f1_model1 = scores_model1["rouge-l"]["f"]

precision_model2 = scores_model2["rouge-l"]["p"]
recall_model2 = scores_model2["rouge-l"]["r"]
f1_model2 = scores_model2["rouge-l"]["f"]

print("Precision, Recall, and F1-score for BLIP-2:")
print("Precision:", precision_model1)
print("Recall:", recall_model1)
print("F1-score:", f1_model1)

print("Precision, Recall, and F1-score for VitGpt:")
print("Precision:", precision_model2)
print("Recall:", recall_model2)
print("F1-score:", f1_model2)

ROUGE Scores for BLIP-2:
{'rouge-1': {'r': 0.7057404605198723, 'p': 0.5711694838529204, 'f': 0.6259112389882882}, 'rouge-2': {'r': 0.46176681935195857, 'p': 0.35838442697653206, 'f': 0.39949114886886683}, 'rouge-l': {'r': 0.6315150650003591, 'p': 0.5119797835463471, 'f': 0.560635915103244}}
ROUGE Scores for VitGpt:
{'rouge-1': {'r': 0.44236701370524906, 'p': 0.4042870038458274, 'f': 0.41503899617383466}, 'rouge-2': {'r': 0.20881120625160865, 'p': 0.16989874025183618, 'f': 0.18336638637340974}, 'rouge-l': {'r': 0.39817976304741015, 'p': 0.3620736453089395, 'f': 0.37246338708799215}}
Precision, Recall, and F1-score for BLIP-2:
Precision: 0.5119797835463471
Recall: 0.6315150650003591
F1-score: 0.560635915103244
Precision, Recall, and F1-score for VitGpt:
Precision: 0.3620736453089395
Recall: 0.39817976304741015
F1-score: 0.37246338708799215
